# Project 3 - Part 4

Author: Nicholas Orgel
    
Date: 03/15/2023

## Questions to Answer
The stakeholder's first question is: does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?

They want you to perform a statistical test to get a mathematically-supported answer.
They want you to report if you found a significant difference between ratings.
If so, what was the p-value of your analysis?
And which rating earns the most revenue?
They want you to prepare a visualization that supports your finding.
It is then up to you to think of 2 additional hypotheses to test that your stakeholder may want to know.

**Some example hypotheses you could test:**

- Do movies that are over 2.5 hours long earn more revenue than movies that are 1.5 hours long (or less)?
- Do movies released in 2020 earn less revenue than movies released in 2018?
- How do the years compare for movie ratings?
- Do some movie genres earn more revenue than others?
- Are some genres higher rated than others?
- etc.

## Import all necessary libraries to conduct ETL

In [16]:
# import main libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# json
import json
import pymysql
pymysql.install_as_MySQLdb()

#sqlalchemy
from sqlalchemy.types import *
from sqlalchemy_utils import create_database, database_exists
from sqlalchemy import create_engine

# scipy
from scipy import stats

import gzip

# External Sources

Github Users: Yvon Bilodeau & Chastity Benton

Code lines will be commented to show where influenced from this user was incorperated. Labeled as XSYB or XSCB

Cited Sources: 

   - Yvon Bilodeau (https://github.com/YBilodeau/Movie-Production-Business-Analysis/blob/main/Phase%204%20-%20TMDB%20ETL.ipynb)

- Chastity Benton (https://github.com/ChastityB/How-to-Make-The-Best-Movies/blob/main/Part%204%20-%20IMDB%20-%20How%20to%20Make%20The%20Best%20Movies%20-%20Hypothesis%20Testing.ipynb)

In [4]:
# XSYB
# Import TMDB data from year 2000
year = 2000
df = pd.read_csv(f'Data/final_tmdb_data_{year}.csv.gz')
df.head(2)

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.4,21.0,NaN


In [8]:
# XSYB
## Remove null values from initial row
df = df.loc[1:,:]

df.head(1)

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.4,21.0,NaN


In [15]:
# Create a for loop that iterates through the different final_tmdb_data years
# XSCB

years = [*range(2001, 2022, 1)]

for year in years:
    # Import TMDB data csv file for current year
    current_year = pd.read_csv(f'Data/final_tmdb_data_{year}.csv.gz', compression='gzip', lineterminator='\n')
    # remove null values from first row in selected data
    current_year = current_year.loc[1:,:]
    # append TMDB data csv file for current year into combined df
    df = df.append(current_year)
    

C:\Users\njfor\AppData\Local\Temp\ipykernel_14212\4161941394.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(current_year)
C:\Users\njfor\AppData\Local\Temp\ipykernel_14212\4161941394.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(current_year)
C:\Users\njfor\AppData\Local\Temp\ipykernel_14212\4161941394.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(current_year)
C:\Users\njfor\AppData\Local\Temp\ipykernel_14212\4161941394.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(current_year)


BadGzipFile: Not a gzipped file (b'\n\n')